# 4 - Feras Formidáveis

## 4.1 Quem classifica a classe classificadora?

### Introdução:

O objetivo desta atividade é alterar uma rede neural regressora feita em Python puro para resolver um problema de classificação. Treinou-se uma rede neural em um dataset simples sobre flores, utilizando a *binary cross entropy* como função de perda e demonstrando o funcionamento dessa rede. Boa parte do algoritmo foi adaptada do material de aula do Daniel Cassar [1]

### Desenvolvimento:

Importando as bibliotecas necessárias [2-5]:

In [1]:
import math
import random
import pandas as pd
import seaborn as sns

A nossa rede neural é baseada em uma estrutura de dados chamada **Valor** [1], em que definimos as operações matemáticas necessárias para o treinamento do modelo e o cálculo do gradiente. Adicionou-se a operação *log*, necessária para o cálculo da função de perda:

In [2]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        return f"Valor(data={self.data})"
    
    def __add__(self, outro_valor):
        """Realiza a operação: self + outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado
    
    def __mul__(self, outro_valor):
        """Realiza a operação: self * outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data # grad_filho * derivada filho em relação a mãe
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado
    
    def exp(self):
        """Realiza a operação: exp(self)"""
        progenitor = (self, )
        data = math.exp(self.data)
        operador_mae = "exp"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado

    def log(self):
        """Realiza a operação: ln(self)"""
        progenitor = (self, )
        data = math.log(self.data)
        operador_mae = "log"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_log():
            self.grad += resultado.grad / (self.data)
        
        resultado.propagar = propagar_log
        
        return resultado

    def __pow__(self, expoente):
        """Realiza a operação: self ** expoente"""
        assert isinstance(expoente, (int, float))
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado
    
    def __truediv__(self, outro_valor):
        """Realiza a operação: self / outro_valor"""
        return self * outro_valor ** (-1)
    
    def __neg__(self):
        """Realiza a operação: -self"""
        return self * -1
    
    def __sub__(self, outro_valor):
        """Realiza a operação: self - outro_valor"""
        return self + (-outro_valor)
    
    def __radd__(self, outro_valor):
        """Realiza a operação: outro_valor + self"""
        return self + outro_valor
    
    def __rmul__(self, outro_valor):
        """Realiza a operação: outro_valor * self"""
        return self * outro_valor
    
    def sig(self):
        """Realiza a operação: exp(self) / (exp(self) + 1)"""
        return self.exp() / (self.exp() + 1)
    
    def propagar(self):
        pass
    
    def propagar_tudo(self):
        
        self.grad = 1
        
        ordem_topologica = []
        
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)
        
        for vertice in reversed(ordem_topologica):
            vertice.propagar()

Nossa rede é composta por neurônios, em que recebe informações de entrada, aplica uma função de ativação e passa a informação adiante, sendo definida na classe Neuronio [1]

In [3]:
class Neuronio:
    def __init__(self, num_dados_entrada):
        self.vies = Valor(random.uniform(-1, 1))
        
        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor(random.uniform(-1, 1)))
            
    def __call__(self, x):
        
        assert len(x) == len(self.pesos)
        
        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno
            
        soma += self.vies  
        dado_de_saida = soma.sig()
        
        return dado_de_saida       
    
    def parametros(self):
        return self.pesos + [self.vies]

Definindo a classe Camada [1], em que os neurônios são agrupados e passando a informação através de camadas ocultas:

In [4]:
class Camada:
    def __init__(self, num_neuronios, num_dados_entrada):
        neuronios = []
        
        for _ in range(num_neuronios):
            neuronio = Neuronio(num_dados_entrada)
            neuronios.append(neuronio)
            
        self.neuronios = neuronios     
        
    def __call__(self, x):
        dados_de_saida = []
        
        for neuronio in self.neuronios:
            informacao = neuronio(x)
            dados_de_saida.append(informacao)
            
        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:        
            return dados_de_saida  
    
    def parametros(self):
        params = []
        
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)
        
        return params

Por fim, definindo nossa rede neural classificadora, que vai organizar nossos dados em camadas 

In [5]:
class Classifier_nn:
    def __init__(self, num_dados_entrada, num_neuronios_por_camada):
        
        percurso = [num_dados_entrada] + num_neuronios_por_camada
        
        camadas = []
        
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada(num_neuronios_por_camada[i], percurso[i])
            camadas.append(camada)
            
        self.camadas = camadas
        
    def __call__(self, x):
        for camada in self.camadas:
            x = camada(x)
        return x
    
    def parametros(self):
        params = []
        
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)
            
        return params

Carregando o dataset próprio da biblioteca *Seaborn* relacionado a plantas [6], contendo características como comprimento e largura das pétalas e sépalas de uma flor, indicando a espécie da planta

In [6]:
df = sns.load_dataset('iris')
logica = df["species"] != 'versicolor'
df_binario = df.loc[logica]
df_binario

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


Utilizou-se apenas 2 das 3 espécies fornecidas no dataset (*setosa* e *virginica*) para esse problema de classificação, convertendo as espécies para número, em que setosa representa 0 e virginica 1

In [7]:
for especie in df_binario['species']:
    if especie == 'setosa':
        df_binario['species'] = df_binario['species'].replace(especie, 0)
    elif especie == 'virginica':
        df_binario['species'] = df_binario['species'].replace(especie, 1)

df_binario

C:\Users\enzo24035\AppData\Local\Temp\ipykernel_15460\3480997974.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_binario['species'] = df_binario['species'].replace(especie, 0)
C:\Users\enzo24035\AppData\Local\Temp\ipykernel_15460\3480997974.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_binario['species'] = df_binario['species'].replace(especie, 1)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1
146,6.3,2.5,5.0,1.9,1
147,6.5,3.0,5.2,2.0,1
148,6.2,3.4,5.4,2.3,1


Separando o dataset em dados de entrada e os valores reais, a fim de comparar com as previsões do modelo, além de definir uma rede neural classificadora com base na arquitetura da rede escolhida. De acordo com o dataset, há 4 atributos do problema, sendo os dados de entrada, e o algoritmo tem 1 dado de saída contendo a previsão da espécie (valor entre 0 e 1, sendo setosa mais próximo de 0 e virginica mais próximo de 1). Note que o valor real *y_true* foi atribuido na classe Valor, para permitir calcular a função de perda e os gradientes locais.

In [8]:
x = df_binario.drop("species", axis=1).values
y_true = df_binario['species'].values
y_true = [Valor(value) for value in y_true]

NUM_DADOS_DE_ENTRADA = 4  
NUM_DADOS_DE_SAIDA = 1    
CAMADAS_OCULTAS = [3, 2]  

arquitetura_da_rede = CAMADAS_OCULTAS + [NUM_DADOS_DE_SAIDA]

meu_classificador = Classifier_nn(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede)

Fazendo a etapa de treinamento para um número de épocas como 50 e uma taxa de aprendizado de 0.001. A função de perda usada é a *binary cross entropy*, própria para problemas de classificação [7], sendo que a cada época a perda é mostrada

In [9]:
NUM_EPOCAS = 50
TAXA_DE_APRENDIZADO = 0.01

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = []
    for exemplo in x:
        previsao = meu_classificador(exemplo)
        y_pred.append(previsao)

    # loss
    erros = []
    for yt, yp in zip(y_true, y_pred):
        variavel = 1 + (- yp)
        erro = (yt.data * yp.log() + (1 - yt.data) * variavel.log())
        erros.append(erro)

    loss = sum(erros) / - len(y_pred)
    
    # zero grad
    for p in meu_classificador.parametros():
        p.grad = 0

    # backpropagation
    loss.propagar_tudo()

    # atualiza parâmetros
    for p in meu_classificador.parametros():
        p.data = p.data - p.grad * TAXA_DE_APRENDIZADO

    # mostra resultado (opcional)
    print(epoca, loss.data)

0 0.7710242085820912
1 0.770438906787496
2 0.7698572676417085
3 0.7692792683828066
4 0.7687048863456198
5 0.7681340989619687
6 0.7675668837608963
7 0.7670032183688968
8 0.7664430805101319
9 0.7658864480066438
10 0.7653332987785583
11 0.7647836108442805
12 0.7642373623206891
13 0.7636945314233148
14 0.7631550964665195
15 0.762619035863666
16 0.7620863281272819
17 0.7615569518692157
18 0.761030885800785
19 0.7605081087329281
20 0.7599885995763352
21 0.7594723373415836
22 0.7589593011392641
23 0.7584494701801013
24 0.757942823775068
25 0.7574393413354945
26 0.7569390023731724
27 0.7564417865004509
28 0.7559476734303339
29 0.75545664297656
30 0.754968675053691
31 0.7544837496771862
32 0.7540018469634727
33 0.7535229471300154
34 0.7530470304953749
35 0.752574077479267
36 0.7521040686026149
37 0.7516369844875935
38 0.7511728058576758
39 0.7507115135376675
40 0.7502530884537446
41 0.7497975116334756
42 0.7493447642058534
43 0.7488948274013106
44 0.7484476825517353
45 0.7480033110904863
46 0.7

Percebe-se que embora a perda seja relativamente pequena, o algoritmo não está tão bem otimizado para o problema de classificação, pois a redução é bem gradual.

Mostrando os valores preditos pela rede neural:

In [10]:
y_pred

[Valor(data=0.34073592096375793),
 Valor(data=0.34125800284814306),
 Valor(data=0.34144852139157894),
 Valor(data=0.34268157739681065),
 Valor(data=0.3410502796859636),
 Valor(data=0.3396904489566612),
 Valor(data=0.3414706324628808),
 Valor(data=0.34147213162194706),
 Valor(data=0.34280531453451957),
 Valor(data=0.34253278478933874),
 Valor(data=0.3403168370037436),
 Valor(data=0.34256964050852795),
 Valor(data=0.34236398862033685),
 Valor(data=0.3425277485836222),
 Valor(data=0.33806134678646194),
 Valor(data=0.3380620752590531),
 Valor(data=0.33801568310923874),
 Valor(data=0.3399632279731445),
 Valor(data=0.3395514093375865),
 Valor(data=0.34044785848798664),
 Valor(data=0.34115506961308734),
 Valor(data=0.33967531634479475),
 Valor(data=0.34047322590899237),
 Valor(data=0.3397575110380722),
 Valor(data=0.3440336169601387),
 Valor(data=0.34186610679804447),
 Valor(data=0.34037752220533724),
 Valor(data=0.34087974981740143),
 Valor(data=0.34042702579482453),
 Valor(data=0.3428575008

Note que todos os valores estão bem próximos e em apenas uma categoria, indicando que o modelo só previu dados para o rótulo 1 (espécie virginica), mesmo com os dados de entrada contendo plantas das 2 espécies

### Conclusão:

Foi possível treinar uma rede neural em python puro para tarefas de classificação, embora ela não esteja bem otimizada, aumentando a perda ao longo do treinamento e prevendo apenas um rótulo para os dados do problema. Mesmo assim, foi interessante aprender sobre esse tipo de problema, aumentando meu conhecimento.

### Referências:

[1] CASSAR, Daniel. "ATP-303 NN 4.2 - Notebook MLP.ipynb". Material de Aula, 2025.

[2] Biblioteca Math. https://docs.python.org/3/library/math.html

[3] Biblioteca Random. https://docs.python.org/3/library/random.html

[4] Biblioteca Pandas. https://pandas.pydata.org/docs/user_guide/index.html#user-guide

[5] Biblioteca Seaborn. https://seaborn.pydata.org/

[6] Dataset estudado. https://archive.ics.uci.edu/dataset/53/iris

[7] Função de perda analisada. https://medium.com/ensina-ai/uma-explicação-visual-para-função-de-custo-binary-cross-entropy-ou-log-loss-eaee662c396c